In [1]:
import pandas as pd
import numpy as np
from dataset_functions import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold

# Random Forest

In [ ]:
params = {
    'n_estimators': [100,200,250],
    'class_weigth': ['balanced'],
    'criterion': ['entropy','gini'],
}

model = RandomForestClassifier()

In [ ]:
df, target = get_dataset()
win = target['radiant_win']

In [ ]:
df_tt = teamstats_teamheros_transform(df.copy())
df_tt = feature_selection_transform(df_tt,win,0.01)

Filtering Df:  (game_mode == 2 or game_mode == 22) and game_time > 0 

Dropped:  ['lobby_type', 'chat_len', 'game_mode', 'match_id_hash'] 

Dataframe Shape:  (32153, 242) 

Target shape: (32153, 6)
Hero Id Labels: ['r1_hero_id', 'r2_hero_id', 'r3_hero_id', 'r4_hero_id', 'r5_hero_id', 'd1_hero_id', 'd2_hero_id', 'd3_hero_id', 'd4_hero_id', 'd5_hero_id'] 

Numbers of Heros:  115 

NaN Count:  0 

Single Player Labels: ['r1_kills', 'r1_deaths', 'r1_assists', 'r1_denies', 'r1_gold', 'r1_lh', 'r1_xp', 'r1_health', 'r1_max_health', 'r1_max_mana', 'r1_level', 'r1_x', 'r1_y', 'r1_stuns', 'r1_creeps_stacked', 'r1_camps_stacked', 'r1_rune_pickups', 'r1_firstblood_claimed', 'r1_teamfight_participation', 'r1_towers_killed', 'r1_roshans_killed', 'r1_obs_placed', 'r1_sen_placed', 'r2_kills', 'r2_deaths', 'r2_assists', 'r2_denies', 'r2_gold', 'r2_lh', 'r2_xp', 'r2_health', 'r2_max_health', 'r2_max_mana', 'r2_level', 'r2_x', 'r2_y', 'r2_stuns', 'r2_creeps_stacked', 'r2_camps_stacked', 'r2_rune_pickups

In [14]:
scorings = ["roc_auc","accuracy","recall","precision","f1"]
best_model = GridSearchCV(estimator=model,param_grid=params,scoring=scorings,refit="roc_auc",cv=StratifiedKFold(n_splits=5,shuffle=True))

best_model.fit(df_tt,win)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             estimator=RandomForestClassifier(),
             param_grid={'class_weight': ['balanced', None]}, refit='roc_auc',
             scoring=['roc_auc', 'accuracy', 'recall', 'precision', 'f1'])

In [42]:
best_model.cv_results_

{'mean_fit_time': array([ 8.30346155, 10.45910792]),
 'std_fit_time': array([0.13127195, 4.73928405]),
 'mean_score_time': array([0.21098404, 0.2371273 ]),
 'std_score_time': array([0.00908625, 0.04854317]),
 'param_class_weight': masked_array(data=['balanced', None],
              mask=[False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'params': [{'class_weight': 'balanced'}, {'class_weight': None}],
 'split0_test_roc_auc': array([0.80401126, 0.80402552]),
 'split1_test_roc_auc': array([0.79678552, 0.7937375 ]),
 'split2_test_roc_auc': array([0.80187797, 0.79990605]),
 'split3_test_roc_auc': array([0.79766943, 0.79749953]),
 'split4_test_roc_auc': array([0.79694962, 0.79663819]),
 'mean_test_roc_auc': array([0.79945876, 0.79836136]),
 'std_test_roc_auc': array([0.00294012, 0.0034514 ]),
 'rank_test_roc_auc': array([1, 2], dtype=int32),
 'split0_test_accuracy': array([0.72166071, 0.72119422]),
 'split1_test_accuracy': array([0.71310838, 0.70828798]),
 'split2

In [41]:
cv_results:dict = best_model.cv_results_

print("Best Parameters:\n",best_model.best_params_,'\n')

for scoring in scorings:
    
    mean_roc_label = f"mean_test_{scoring}"

    mean_score_list = cv_results[f"mean_test_{scoring}"]

    best_position = np.argmin(best_model.cv_results_["rank_test_roc_auc"])

    print(f"{scoring}:\n",mean_score_list[best_position])


Best Parameters:
 {'class_weight': 'balanced'} 

roc_auc:
 0.7994587611775285
accuracy:
 0.7157029580427038
recall:
 0.7604584500783813
precision:
 0.7173205662413586
f1:
 0.7382118094540092


In [ ]:
depths = [tree.get_depth() for tree in best_model.best_estimator_.estimators_]
print(np.max(depths))
print(np.min(depths))
print(np.mean(depths))

41
27
32.89


In [34]:
feature_importance = {
    name: value 
    for name,value in zip(best_model.best_estimator_.feature_names_in_,best_model.best_estimator_.feature_importances_)
}

feature_importance = dict(reversed(sorted(feature_importance.items(), key=lambda item: item[1])))
feature_names = list(feature_importance.keys())
print("Most Important:\n", feature_names[:10])
print("Least Important:\n", feature_names[-10:])


Most Important:
 ['d_gold', 'r_kills', 'd_kills', 'd_deaths', 'd_health', 'r_deaths', 'r1_y', 'r_gold', 'r_denies', 'r_health']
Least Important:
 ['r4_x', 'd3_x', 'd2_x', 'd_assists', 'r_towers_killed', 'd4_y', 'r_assists', 'game_time', 'r_level', 'd_level']
